This dataset includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD) from 2006 to the end of last year (2019). For additional details, please see the attached data dictionary in the ‘About’ section.

link: https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [1]:
# !pip install -U scikit-learn
# !pip install xgboost==0.81
# !pip install sodapy
#  Example authenticated client (needed for non-public datasets):
# https://data.cityofnewyork.us/resource/qgea-i56i.json
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:

# from sodapy import Socrata
# client = Socrata("data.cityofnewyork.us", None)

# results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# results = client.get("qgea-i56i", limit=750000)


# Convert to pandas DataFrame
# results_df = pd.DataFrame.from_records(results)

## 1 - Extração e carregamento dos dados.

- Importando as bibliotecas.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import datetime
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns

- Solicitando os dados por meio da API sodapy

In [2]:
url='https://data.cityofnewyork.us/resource/qgea-i56i.json?$limit=750000'

In [3]:
dados = pd.read_json(url)

- Exibindo o daatset

In [4]:
pd.set_option("display.max_columns", 50)
dados.head()

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,addr_pct_cd,boro_nm,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_num,cmplnt_to_dt,cmplnt_to_tm,crm_atpt_cptd_cd,hadevelopt,housing_psa,juris_desc,jurisdiction_code,ky_cd,lat_lon,latitude,law_cat_cd,loc_of_occur_desc,longitude,ofns_desc,parks_nm,patrol_boro,pd_cd,pd_desc,prem_typ_desc,rpt_dt,station_name,susp_age_group,susp_race,susp_sex,transit_district,vic_age_group,vic_race,vic_sex,x_coord_cd,y_coord_cd
0,29.0,10930.0,6.0,29.0,5.0,46.0,BRONX,2015-05-28T00:00:00.000,15:00:00,700381962,NaN,NaN,COMPLETED,NaN,NA,N.Y. POLICE DEPT,0.0,578,"{'latitude': '40.84586773', 'longitude': '-73....",40.845868,VIOLATION,INSIDE,-73.915888,HARRASSMENT 2,NaN,PATROL BORO BRONX,638.0,"HARASSMENT,SUBD 3,4,5",RESIDENCE - APT. HOUSE,2015-06-01T00:00:00.000,NaN,25-44,BLACK,M,NaN,25-44,WHITE HISPANIC,F,1007522.0,247458.0
1,13.0,10691.0,4.0,74.0,1.0,120.0,STATEN ISLAND,2013-10-28T00:00:00.000,13:50:00,642234217,2013-10-28T00:00:00.000,13:50:00,COMPLETED,NaN,NA,N.Y. POLICE DEPT,0.0,351,"{'latitude': '40.627060894', 'longitude': '-74...",40.627061,MISDEMEANOR,INSIDE,-74.077149,CRIMINAL MISCHIEF & RELATED OF,NA,PATROL BORO STATEN ISLAND,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",RESIDENCE - APT. HOUSE,2013-10-28T00:00:00.000,NaN,NaN,NaN,NaN,NaN,45-64,WHITE,M,962835.0,167738.0
2,19.0,12422.0,20.0,15.0,4.0,24.0,MANHATTAN,2012-05-09T00:00:00.000,20:50:00,242465164,2012-05-09T00:00:00.000,21:00:00,COMPLETED,NaN,NA,N.Y. POLICE DEPT,0.0,236,"{'latitude': '40.800965968', 'longitude': '-73...",40.800966,MISDEMEANOR,NaN,-73.969047,DANGEROUS WEAPONS,NA,PATROL BORO MAN NORTH,782.0,"WEAPONS, POSSESSION, ETC",STREET,2012-05-09T00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,E,992820.0,231089.0
3,33.0,14787.0,53.0,66.0,3.0,108.0,QUEENS,2014-01-03T00:00:00.000,13:30:00,927207428,2014-01-03T00:00:00.000,13:35:00,ATTEMPTED,NaN,NA,N.Y. POLICE DEPT,0.0,109,"{'latitude': '40.745241809', 'longitude': '-73...",40.745242,FELONY,INSIDE,-73.894253,GRAND LARCENY,NaN,PATROL BORO QUEENS NORTH,409.0,"LARCENY,GRAND BY EXTORTION",RESIDENCE - APT. HOUSE,2014-01-03T00:00:00.000,NaN,NaN,UNKNOWN,M,NaN,45-64,ASIAN / PACIFIC ISLANDER,M,1013552.0,210803.0
4,35.0,10932.0,49.0,23.0,5.0,40.0,BRONX,2016-04-13T00:00:00.000,00:00:00,492142357,NaN,NaN,COMPLETED,NaN,NA,N.Y. POLICE DEPT,0.0,351,"{'latitude': '40.810351863', 'longitude': '-73...",40.810352,MISDEMEANOR,NaN,-73.924942,CRIMINAL MISCHIEF & RELATED OF,NaN,PATROL BORO BRONX,258.0,"CRIMINAL MISCHIEF 4TH, GRAFFIT",HIGHWAY/PARKWAY,2016-04-13T00:00:00.000,NaN,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN,E,1005028.0,234516.0


- Selecionando apenas os dados correspondentes a jurisdição da polícia.

In [5]:
dados = dados.query('jurisdiction_code==0')

In [6]:
dados.shape

(664388, 40)

## 2 - Tratamento dos dados.

- Neste tópico será feito um tratamento de dados, com exclusão, modificação do  tipo e conteúdo de algumas colunas.


- Criando a lista de colunas a serem excluídas.

In [7]:
drop_columns=['cmplnt_num','addr_pct_cd','ky_cd','pd_cd','juris_desc','jurisdiction_code','housing_psa','x_coord_cd','y_coord_cd','lat_lon',':@computed_region_efsh_h5xi',
              ':@computed_region_f5dn_yrer',':@computed_region_yeji_bk3q',':@computed_region_92fq_4b7q',':@computed_region_sbqj_enih','susp_age_group','susp_race','susp_sex',
              'patrol_boro','hadevelopt']

- Excluindo as colunas.

In [8]:
dados.drop(columns=drop_columns,axis=1,inplace=True)

- Reordenando as colunas do dataset.

In [9]:
dados.columns=['bairro','data_da_ocorr','hora_da_ocorr','data_final_ocorr','hora_termino_ocorr',
                'conclusao','latitude','nivel_delito','loc_esp_delito','longitude',
                'delito','parque','desc_delito','desc_instalacoes','data_com_policia',
               'rua_av','transit_district','idade_vitima','raca_vitima','sexo_vitima']

In [10]:
dados.head()

,bairro,data_da_ocorr,hora_da_ocorr,data_final_ocorr,hora_termino_ocorr,conclusao,latitude,nivel_delito,loc_esp_delito,longitude,delito,parque,desc_delito,desc_instalacoes,data_com_policia,rua_av,transit_district,idade_vitima,raca_vitima,sexo_vitima
0,BRONX,2015-05-28T00:00:00.000,15:00:00,NaN,NaN,COMPLETED,40.845868,VIOLATION,INSIDE,-73.915888,HARRASSMENT 2,NaN,"HARASSMENT,SUBD 3,4,5",RESIDENCE - APT. HOUSE,2015-06-01T00:00:00.000,NaN,NaN,25-44,WHITE HISPANIC,F
1,STATEN ISLAND,2013-10-28T00:00:00.000,13:50:00,2013-10-28T00:00:00.000,13:50:00,COMPLETED,40.627061,MISDEMEANOR,INSIDE,-74.077149,CRIMINAL MISCHIEF & RELATED OF,NA,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",RESIDENCE - APT. HOUSE,2013-10-28T00:00:00.000,NaN,NaN,45-64,WHITE,M
2,MANHATTAN,2012-05-09T00:00:00.000,20:50:00,2012-05-09T00:00:00.000,21:00:00,COMPLETED,40.800966,MISDEMEANOR,NaN,-73.969047,DANGEROUS WEAPONS,NA,"WEAPONS, POSSESSION, ETC",STREET,2012-05-09T00:00:00.000,NaN,NaN,NaN,UNKNOWN,E
3,QUEENS,2014-01-03T00:00:00.000,13:30:00,2014-01-03T00:00:00.000,13:35:00,ATTEMPTED,40.745242,FELONY,INSIDE,-73.894253,GRAND LARCENY,NaN,"LARCENY,GRAND BY EXTORTION",RESIDENCE - APT. HOUSE,2014-01-03T00:00:00.000,NaN,NaN,45-64,ASIAN / PACIFIC ISLANDER,M
4,BRONX,2016-04-13T00:00:00.000,00:00:00,NaN,NaN,COMPLETED,40.810352,MISDEMEANOR,NaN,-73.924942,CRIMINAL MISCHIEF & RELATED OF,NaN,"CRIMINAL MISCHIEF 4TH, GRAFFIT",HIGHWAY/PARKWAY,2016-04-13T00:00:00.000,NaN,NaN,UNKNOWN,UNKNOWN,E


- Observando dados asentes e o tipo das colunas, para a partir daí tratar os dados faltantes e alterar o tipo das colunas que for necesário.

In [11]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664388 entries, 0 to 749999
Data columns (total 20 columns):
bairro                664386 non-null object
data_da_ocorr         664371 non-null object
hora_da_ocorr         664388 non-null object
data_final_ocorr      551693 non-null object
hora_termino_ocorr    551980 non-null object
conclusao             664388 non-null object
latitude              664295 non-null float64
nivel_delito          664388 non-null object
loc_esp_delito        537930 non-null object
longitude             664295 non-null float64
delito                663734 non-null object
parque                147893 non-null object
desc_delito           664388 non-null object
desc_instalacoes      661845 non-null object
data_com_policia      664388 non-null object
rua_av                31 non-null object
transit_district      31 non-null float64
idade_vitima          587700 non-null object
raca_vitima           664385 non-null object
sexo_vitima           664385 non-null o

### 2.1 **Criando novas colunas a partir da data da ocorrência.**

- Excluindo dados ausentes referentes a coluna data da ocorrência.

In [12]:
df = dados.dropna(subset=['data_da_ocorr'])

- Criando novas variáveis referentes a data da ocorrencia, como dia da semana, mês, ano e dia do mês.

In [13]:
def date_to_weekday(date):
    weekday_dict = {0:'segunda', 1:'terca', 2: 'quarta', 3: 'quinta', 4: 'sexta', 5: 'sabado', 6: 'domingo'}
    date_time_obj = datetime.datetime.strptime(date, '%Y-%m-%d')
    return weekday_dict[date_time_obj.weekday()]

In [14]:
date = df['data_da_ocorr'].str.split('T',expand=True, n=1)[0]
date_1 = date.str.split("-", n = 3, expand = True)
df['ano'] = date_1[0].astype('int32')
df['mes'] = date_1[1].astype('int32')
df['dia'] = date_1[2].astype('int32')
df['dia_da_semana'] = date.apply(date_to_weekday)

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

- Criando a variável hora da ocorrência.

In [15]:
df['hora'] = df['hora_da_ocorr'].str.split(':',n=2,expand=True)[0].astype(int)

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


-  **Alterando a ordem das colunas do dataframe**

In [16]:
df = df[['data_da_ocorr', 'hora_da_ocorr',
      'conclusao','delito','desc_delito','nivel_delito',
       'loc_esp_delito','desc_instalacoes','bairro','latitude','longitude',  
       'hora','dia', 'mes', 'ano','dia_da_semana','idade_vitima','raca_vitima','sexo_vitima']]

- Convertendo a coluna "**data_da_ocorr**"  pro formato datetime 

In [17]:
df['data_da_ocorr'] = pd.to_datetime(df['data_da_ocorr'], format=r'%Y-%m-%d', errors='coerce')

- Verificando e removendo dados inconsistentes com relação a data.

In [18]:
display(df['data_da_ocorr'].dt.year.min())
display(df['data_da_ocorr'].dt.year.max())

1912.0

2019.0

In [19]:
df = df[df['ano'] >= 2006]

- Traduzindo os dados das colunas "**conclusao**", "**nivel_delito**" e "**loc_esp_delito**".

In [20]:
#  alterando os dados de  select_ny['conclusao']
new_conclusao = {'ATTEMPTED': 0, 'COMPLETED': 1 }
df['conclusao'] = df['conclusao'].map(new_conclusao)

# alterando os dados de  df['nivel_de_ofensa']
new_nivel_delito = {'MISDEMEANOR': "contravencao", 'FELONY': "crime", 'VIOLATION': "violacao" }
df['nivel_delito'] = df['nivel_delito'].map(new_nivel_delito)


# alterando os dados de  df['local_da_ocorrencia'], dentro ou fora dos estabelecimentos 
new_local_esp_delito = {'INSIDE': "dentro", 'FRONT OF': "fora", 'OPPOSITE OF': "fora", 
                           'REAR OF': "fora", 'OUTSIDE': "fora" }
df['loc_esp_delito'] = df['loc_esp_delito'].map(new_local_esp_delito)

# alterando os dados da coluna bairro

new_bairro = {'BRONX': 'Bronx', 'STATEN ISLAND': 'Staten_Island', 'BROOKLYN': 'Brooklyn', 'MANHATTAN': 'Manhattan' , 'QUEENS': 'Queens' }
df['bairro'] = df['bairro'].map(new_bairro)

## 3 - Criando novas features.

 - ### Criando a variável noite.
 
 Variável que vai ajudar o algoritmo na hora da modelagem na associação com a hora do delito com período do dia.

In [21]:
df['noite'] = df['hora_da_ocorr'].apply(lambda x: 0 if x <'17:59:59' and x >'05:59:59' else 1)


 - ### Criando a variável assalto.
 
 
 O objetivo final é fazer a previsão dessa variável, que quando igual a 1 representa ocorrência do crime de assalto e quando igual a 0 significa que não aconteceu um delito classificado com assalto.
    
    
    
Em uma primeira análise classifiquei os delitos abaixo com sendo crimes de assalto :
 
   "ROBBERY","PETIT LARCENY","GRAND LARCENY","GRAND LARCENY OF MOTOR VEHICLE","ASSAULT 3 & RELATED OFFENSES","FELONY ASSAULT"
   

Após identificar os delitos, crei uma função para criar a variável assalto.

In [22]:
lista_assalto = ['PETIT LARCENY','GRAND LARCENY OF MOTOR VEHICLE', 'ROBBERY',
                 'GRAND LARCENY', 'UNAUTHORIZED USE OF A VEHICLE',
                 "BURGLARY","BURGLAR'S TOOLS","POSSESSION OF STOLEN PROPERTY","PETIT LARCENY OF MOTOR VEHICLE"]

df['assalto'] = df['delito'].apply(lambda x: 1 if x in lista_assalto else 0)

In [23]:
df.assalto.value_counts(normalize=True)

0    0.625961
1    0.374039
Name: assalto, dtype: float64

- FILTRANDO OS DELITOS CONSIDERADOS COMO ASSALTO.


  - Após fazer um apanhado geral dos delitos que seriam assalto e ter criado a variável na célula acima, fiz um filtro mais minucioso por meio da variável "desc_delito"(que retorna a descrição de cada delito cometido) e observei que vários delitos classificados como sendo assalto não tem a caricterística de um crime de assalto(crimes de fraude, agressão física), sendo necessário fazer outro filtro e alterar o valor desses registros que realmente não são assalto.

In [24]:

lista = ["MENACING,UNCLASSIFIED","OBSTR BREATH/CIRCUL",
         "ASSAULT SCHOOL SAFETY AGENT","ASSAULT TRAFFIC AGENT","STRANGULATION 1ST",
         "ROBBERY, PAYROLL","LARCENY,PETIT BY FALSE PROMISE","LARCENY,PETIT BY CREDIT CARD U",
         "LARCENY,PETIT FROM BUILDING,UNATTENDED, PACKAGE THEFT OUTSIDE","LARCENY,PETIT OF BOAT","LARCENY,PETIT FROM BOAT",
         "LARCENY,PETIT FROM PARKING MET","PETIT LARCENY OF ANIMAL","LARCENY,PETIT BY ACQUIRING LOS",
         "LARCENY,PETIT BY DISHONEST EMP","PETIT LARCENY-CHECK FROM MAILB","LARCENY,GRAND BY CREDIT CARD COMPROMISE-UNAUTHORIZE PURCHASE","LARCENY,GRAND OF BOAT",
         "LARCENY,GRAND FROM COIN MACHIN","LARCENY,GRAND FROM PIER, UNATTENDED","LARCENY,GRAND FROM BOAT, UNATTENDED","GRAND LARCENY-CHECK FROM MAILB",
         "LARCENY,GRAND FROM RESIDENCE/BUILDING,UNATTENDED, PACKAGE THEFT OUTSIDE","LARCENY,GRAND BY FALSE PROMISE","LARCENY,GRAND BY BANK ACCT COMPROMISE-UNAUTHORIZED PURCHASE",
         "LARCENY,GRAND BY ACQUIRING LOS","LARCENY,GRAND BY DISHONEST EMP","LARCENY,GRAND BY BANK ACCT COMPROMISE-ATM TRANSACTION","LARCENY,GRAND BY EXTORTION",
         "LARCENY,GRAND BY OPEN/COMPROMISE CELL PHONE ACCT","LARCENY,GRAND BY ACQUIRING LOST CREDIT CARD","LARCENY,GRAND BY OPEN CREDIT CARD (NEW ACCT)",
         "LARCENY,GRAND BY BANK ACCT COMPROMISE-REPRODUCED CHECK","LARCENY,GRAND BY CREDIT CARD ACCT COMPROMISE-EXISTING ACCT","LARCENY,GRAND BY FALSE PROMISE-NOT IN PERSON CONTACT",
         "LARCENY,GRAND BY THEFT OF CREDIT CARD",""]


df_assalto = df[df['assalto']==1]
df_assalto = df_assalto.drop('assalto',axis=1)

df_assalto['assalto'] = df['desc_delito'].apply(lambda x: 0 if x in lista else 1)

- ANALISANDO OS DELITOS QUE NÃO FORAM CONSIDERADOS COMO ASSALTO.


- Fiz uma análise dos delitos que não tiveram uma primeira classificação coo sendo de crime de assalto e encontrei um registro que também deve ser alterado.

In [25]:
df_sem_assalto = df[df['assalto']==0]

In [26]:
lista = "UNAUTHORIZED USE VEHICLE 3"


df_sem_assalto = df_sem_assalto.drop('assalto',axis=1)
df_sem_assalto['assalto'] = df_sem_assalto['desc_delito'].apply(lambda x: 1 if x==lista else 0)

In [27]:
df_completo = pd.concat([df_sem_assalto,df_assalto])

- Excluindo os delitos com poucos registros.

In [28]:
drop = list(dict(df_completo.delito.value_counts().tail(11)).keys())
df_completo = df_completo.set_index('delito')
df_completo = df_completo.drop(drop)
df_completo = df_completo.reset_index()

In [29]:
drop = list(dict(df_completo.desc_instalacoes.value_counts().tail(3)).keys())
df_completo = df_completo.set_index('desc_instalacoes')
df_completo = df_completo.drop(drop)
df_completo = df_completo.reset_index()

In [30]:
df_completo.assalto.value_counts(normalize=True)

0    0.664639
1    0.335361
Name: assalto, dtype: float64

In [31]:
df_completo = df_completo.reset_index(drop=True)

 - ### Criando a variável final de semana.

In [32]:
df_completo['fds'] = df_completo['dia_da_semana'].apply(lambda x: 1 if x=='sabado' or x =='domigo' else 0)

- ### Tratando as colunas referente a vítima.

In [33]:
df_completo = df_completo.query('sexo_vitima == "F" or sexo_vitima=="M"')

In [40]:
df_completo = df_completo.dropna()

In [41]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400676 entries, 0 to 490505
Data columns (total 22 columns):
desc_instalacoes    400676 non-null object
delito              400676 non-null object
data_da_ocorr       400676 non-null datetime64[ns]
hora_da_ocorr       400676 non-null object
conclusao           400676 non-null int64
desc_delito         400676 non-null object
nivel_delito        400676 non-null object
loc_esp_delito      400676 non-null object
bairro              400676 non-null object
latitude            400676 non-null float64
longitude           400676 non-null float64
hora                400676 non-null int64
dia                 400676 non-null int32
mes                 400676 non-null int32
ano                 400676 non-null int32
dia_da_semana       400676 non-null object
idade_vitima        400676 non-null object
raca_vitima         400676 non-null object
sexo_vitima         400676 non-null object
noite               400676 non-null int64
assalto             40067

In [42]:
df_completo = df_completo.sort_values('data_da_ocorr', ascending=True)

In [43]:
df_completo = df_completo.reset_index(drop=True)

In [44]:
df_completo.head()

,desc_instalacoes,delito,data_da_ocorr,hora_da_ocorr,conclusao,desc_delito,nivel_delito,loc_esp_delito,bairro,latitude,longitude,hora,dia,mes,ano,dia_da_semana,idade_vitima,raca_vitima,sexo_vitima,noite,assalto,fds
0,COMMERCIAL BUILDING,THEFT-FRAUD,2006-01-01,00:01:00,1,"FRAUD,UNCLASSIFIED-FELONY",crime,dentro,Brooklyn,40.602071,-74.002692,0,1,1,2006,domingo,25-44,WHITE,M,1,0,0
1,RESIDENCE - APT. HOUSE,SEX CRIMES,2006-01-01,08:00:00,1,"SEXUAL ABUSE 3,2",contravencao,dentro,Brooklyn,40.680580,-73.974317,8,1,1,2006,domingo,<18,UNKNOWN,M,0,0,0
2,RESIDENCE - APT. HOUSE,FRAUDS,2006-01-01,00:01:00,1,"FRAUD,UNCLASSIFIED-MISDEMEANOR",contravencao,dentro,Brooklyn,40.675044,-73.896676,0,1,1,2006,domingo,45-64,WHITE HISPANIC,M,1,0,0
3,RESIDENCE - APT. HOUSE,THEFT-FRAUD,2006-01-01,10:00:00,1,"FRAUD,UNCLASSIFIED-FELONY",crime,dentro,Brooklyn,40.661964,-73.889912,10,1,1,2006,domingo,65+,BLACK,M,0,0,0
4,RESIDENCE-HOUSE,SEX CRIMES,2006-01-01,00:00:00,1,"SEXUAL ABUSE 3,2",contravencao,dentro,Bronx,40.856163,-73.844397,0,1,1,2006,domingo,25-44,BLACK,F,1,0,0


## 4 - Salvando o dataset.

In [45]:
df_completo.columns

Index(['desc_instalacoes', 'delito', 'data_da_ocorr', 'hora_da_ocorr',
       'conclusao', 'desc_delito', 'nivel_delito', 'loc_esp_delito', 'bairro',
       'latitude', 'longitude', 'hora', 'dia', 'mes', 'ano', 'dia_da_semana',
       'idade_vitima', 'raca_vitima', 'sexo_vitima', 'noite', 'assalto',
       'fds'],
      dtype='object')

In [46]:
df_completo.to_csv('delitos_novo_nyc.csv',index=False)